In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.api as sm
from sklearn.datasets import load_boston
import matplotlib.font_manager as fm
import scipy as sp



# # GPU 설정 : 런타임 > 런타임 유형 변경 > GPU
# # GPU 사용 가능한지 확인
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## 잔차 정규성

데이터가 모형 가정을 만족하면 분석결과로 나온 잔차는 정규분포를 따라야 한다. 
예를 들어 다음과 같이 2차 비선형 관계를 가지는 데이터를 선형 모형으로 회귀 분석한 경우를 살펴보자.



In [ ]:
def make_regression2(n_sample=100,bias=0,noise=0.3, random_state=0):
    np.random.seed(random_state)
    x=np.random.rand(n_sample)*4
    epsilon=noise*np.random.randn(n_sample)
    y=x**2+bias+epsilon
    return x,y

x2,y2=make_regression2()
plt.scatter(x2,y2)
plt.show()


In [ ]:
dfX20=pd.DataFrame(x2,columns=["X1"])
dfX2=sm.add_constant(dfX20)
dfY2=pd.DataFrame(y2,columns=["y"])

model2=sm.OLS(dfY2,dfX2)
result2=model2.fit()

In [ ]:
sp.stats.probplot(result2.resid,plot=plt)
plt.show()

In [ ]:
# QQ플롯과 정규성 검정에서 보듯이 잔차는 정규 분포를 따르지 않는다.
test=sm.stats.omni_normtest(result2.resid)
[print("%-12s : %6.3f" % xi) for xi in zip(["Chi^2","P-value"],test)]

### 잔차와 독립 변수의 관계
이 데이터는 2차 함수 관계를 따르고 있으므로 선형 모형을 따르지 않는다.


데이터가 모형 가정을 따르지 않지만 잔차는 정규 분포를 따르는 것을 알 수 있다.
이러한 경우에는 잔차와 독립 변수간의 관계를 살펴보는 것이 도움이 될 수 있다. 
데이터가 올바른 모형으로 분석되었다면 잔차는 독립 변수와 상관관계를 가지지 않아야 한다. 
만약 잔차와 독립 변수간에 어떤 비선형 상관관계를 찾을 수 있다면 올바른 모형이 아니다.

다만 잔차와 특정 독립 변수간의 관계를 전체 모형이 올바른 모형이 아니라는 것을 
알려줄 뿐이지 어떤 모형이 올바른 모형인지에 대한 정보는 주지 않는다.

In [ ]:
def make_regression3(n_sample=100,bias=0,noise=0.5,random_state=0):
    np.random.seed(random_state)
    x=np.random.randn(n_sample)*3-1
    epsilon=noise*np.random.randn(n_sample)
    y=x**2+bias+epsilon
    return x,y

x3,y3=make_regression3()
plt.scatter(x3,y3)
plt.show()

In [ ]:
dfX30=pd.DataFrame(x3,columns=["X1"])
dfX3=sm.add_constant(dfX30)
dfY3=pd.DataFrame(y3,columns=["y"])

model3=sm.OLS(dfY3,dfX3)
result3=model3.fit()

In [ ]:
sp.stats.probplot(result3.resid,plot=plt)
plt.show()

In [ ]:
def chi_square_judge(result):
    test=sm.stats.omni_normtest(result.resid)
    for xi in zip(["Chi^2","P-value"],test):
        if xi[0] !="P-value":
            continue
        elif (xi[-1]<=0.05):
            return "유효합니다."
        elif (xi[-1]>0.05):
            return "기각합니다."

chi_square_judge(result3)

In [ ]:
plt.plot(x3,result3.resid,"o")
plt.axhline(y=0,c="k")
plt.show()

## 이분산성
선형 회귀 모형에서는 종속 변수 값의 분산이 독립 변수의 값과 상관없이 고정된 값을 가져야한다. 그러나 실제 데이터는 독립 변수 값의 크기가 커지면 종속 변수 값의 분산도 커지는 이분산성(heteroskedastic) 문제가 발생한다. 종속변수를 로그 변환한 트랜스로그(translog) 모형을 사용하면 이분산성 문제가 해결되는 경우도 있다.

In [ ]:
from sklearn.datasets import load_diabetes
diabetes=load_diabetes()
diabetes_col=["x%d" % (i) for i in range(diabetes.data.shape[1])]
diab_data=pd.DataFrame(diabetes.data,columns=diabetes_col)
diab_target=pd.DataFrame(diabetes.target,columns=["target"])
diab=pd.merge(diab_data,diab_target,left_index=True,right_index=True)
diab

In [ ]:
model =sm.OLS.from_formula("target ~ x9",data=diab)
result=model.fit()
plt.scatter(diab["x9"],result.resid)
plt.show()